In [2]:
from clean import clean_mi_pl_bs, clean_mi_deferrals, clean_gross_tp_walk, clean_unearned_premium_unexpired_risk, clean_s2_summary_sheet, clean_version_control_solvency_balance_sheet, clean_version_control_scr_review, clean_version_control_s2_gaap, clean_version_control_s2_cap_prov_summary
import pandas as pd
import math
import numpy as np

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
# pd.set_option('display.max_rows', None)
# pd.set_option('display.width', 1000)
# pd.set_option('display.max_colwidth', None)

Import files

In [5]:
all_s2_tables = clean_s2_summary_sheet(file_path = 'documents\\2506 MICL_S2 Gross TP walk.xlsx', sheet_name='S2 Balance Sheet Summary')
scr_summary = all_s2_tables['scr']
mi_bs = clean_mi_pl_bs(file_path = 'documents\\Jun25 MICL MI Pack v3.1 Isi pack exc PC.xlsx', sheet_name='BS')
solvency_balance_sheet_liabilities = clean_version_control_solvency_balance_sheet(file_path = 'documents\\2506 MICL Version Control.xlsx', skip_rows = 55, n_rows = 34, element = 'Liabilities')
# op_risk_calc = all_s2_tables['op_risk']

c:\Users\georg\Desktop\mul_doc\repo\clean.py:17: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  top_header = top_header_raw.ffill().tolist()


In [6]:
mi_bs.head()

,Code,Description,2025-06-01 00:00:00 Actual,2025-06-01 00:00:00 Budget,2025-06-01 00:00:00 Variance,Prior_Year,V to PY,,Comments
0,NaN,Assets,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Heading,Investments,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Heading,Debt securities and other financial investments,23624193.53,2.487288e+07,-1.248683e+06,27334207.66,-3710014.13,NaN,NaN
3,3602,Pluto Investment,672921.57,1.104404e+06,-4.314824e+05,3145025.52,-2472103.95,Investments,Investment in run-off and expected to fully di...
4,3603,Avantus Loan Notes,3112992.16,3.180783e+06,-6.779129e+04,3087007.84,25984.32,Investments,Increased due to valuation gains


### Market Risk

#### Interest Rate Risk

8.2.3 The interest rate risk sub-module considers the **assets stressed within the spread risk sub-module** (i.e. bonds and loans) and the **technical provisions** as those assets and liabilities are exposed to movements in the term structure of interest rates.

8.2.5 The net technical provisions are calculated based on the net expected claims settlements given the claims settlement patterns detailed in Section 4.11.

Maturity (in years): Stress (relative decrease)

1: 75%;
2: 65%;
3: 58%;
4: 52%;
5: 47%;
6: 42%;
7: 38%;
8: 35%;
9: 32%;
10: 30%;
11: 28%;
12: 27%;
13: 26%;
14: 25%;
15: 24%;
16: 23%;
17: 22%;
18: 21%;
19: 20%;
20: 20%;
90: 20%.

Obtaining assets used in spread risk

In [7]:
assets_spread_risk = [  # List from SII Assumptions Doc, but names didn't match up fully
    'CCA Longevity III DAC (USD)',  # 'CCA longevity' in SII Assumptions Doc
    # 'JP Morgan Fund',  # Not in MI pack BS
    # 'Horizon',  # Not in MI pack BS
    'Pluto Investment',  # 'Pluto' in SII Assumptions Doc
    'Avantus Loan Notes',  # 'Avantus' in SII Assumptions Doc
    'Dayim Loan (USD Purchase)',  # 'Dayim' in SII Assumptions Doc
    'Colchis RBLF',  # 'Colchis' in SII Assumptions Doc
    'Hiyacar',
    'HyperJar Investment',  # 'HyperJar' in SII Assumptions Doc
#    'Ibuyer',  # Not in MI pack BS
    'Perceptive',
    'WIL Loan',  # 'Wolvercote loan' in SII Assumptions Doc
    # 'Deep Discounted bonds'  # Not in MI pack BS; 'Vanderbilt Bonds'?
]

df_asset_spread = mi_bs[mi_bs['Description'].isin(assets_spread_risk)][
    ['Description', '2025-06-01 00:00:00 Actual']
].rename(columns={
    'Description': 'Spread Asset',
    '2025-06-01 00:00:00 Actual': 'Value'
}).reset_index(drop=True)

technical_provisions = [
    # 'Provision for unearned premiums, gross amount',
    'Provision unearned premium',
    # 'Claims outstanding, gross amount',  # Is sum of 'Provision Claims outstanding', 'Coin share provision for IBNR Net', 'Coin share claims outstanding', 'IBNR - Gross Amount', 'Provision for IBNR - Net', 'Provision for IBNR - Management load'
    'Provision Claims outstanding',
    'Coin share provision for IBNR Net',
    'Coin share claims outstanding',
    'IBNR - Gross Amount',
    'Provision for IBNR - Net',
    # 'Provision for IBNR - Management load',  # Cannot be included in calculation for SII purposes
    # 'Unexpired risk reserve',
    # 'AURR'
]

df_technical_provisions = mi_bs[mi_bs['Description'].isin(technical_provisions)][
    ['Description', '2025-06-01 00:00:00 Actual']
].rename(columns={
    'Description': 'Technical Provision',
    '2025-06-01 00:00:00 Actual': 'Value'
}).reset_index(drop=True)

# print(np.abs(df_technical_provisions['Value'].sum()))
# display(df_technical_provisions)

In [8]:
ir_risk_provided = scr_summary.loc[scr_summary['Item'] == 'Interest rate', 'Final - As at 30th June 2025'].item()
print(ir_risk_provided)

683872.1253521964


#### Spread Risk

2024 SII Assumptions Doc was used to construct the list. Not all were found in MI Pack Balance sheet. For those found, there is no information on rating, duration or collateralisation status. For the moment, figures are copied from the 2024 SII Assumptions Doc.

For each asset, the SCR is `Market Value × Duration × Risk Factor (based on rating)`. Unrated assets with a duration under 5 years use a 3% risk factor.

What risk factor is associated with each rating?

In [9]:
# Data extracted from SII Assumptions Doc
data_spread = {
    "Spread assets": [
        "CCA longevity", "JP Morgan Fund", "Horizon", "Pluto", "Avantus",
        "Dayim", "Colchis", "Hiyacar", "HyperJar", "Ibuyer", "Perceptive",
        "Wolvercote loan", "Deep Discounted bonds"
    ],
    "£'000s": [
        1440, 26813, 0, 1938, 3087, 5372, 0, 200, 972, 2599, 743, 5336, 9643
    ],
    "Rating": [
        "NR", "AAA", "NR", "NR", "NR", "NR", "NR", "NR", "NR", "NR", "NR", "NR", "NR"
    ],
    "Aver.Duration": [
        1.75, 1.00, 1.00, 1.00, 1.75, 1.75, 1.00, 1.00, 1.00, 1.00, 1.00, 2.81, 2.67
    ],
    "Collateralised": [
        "No", "No", "No", "Yes", "No", "No", "No", "No", "No", "No", "No", "No", "Yes"
    ]
}

# Create a pandas DataFrame from the data
df_spread_risk = pd.DataFrame(data_spread)

df_spread_risk['Value'] = df_spread_risk["£'000s"] * 1000
df_spread_risk = df_spread_risk.drop(columns=["£'000s"])

In [10]:
def calculate_scr(row):

    # 8.3.5: Risk factor for unrated assets with duration < 5 years is 3%. Here, all 'NR' rated assets have a duration < 5. We don't know what risk factor to give 'AAA' - assumet it is 0
    if row['Rating'] == 'NR' and row['Aver.Duration'] < 5:
        risk_factor = 0.03
    if row['Rating'] == 'AAA' and row['Aver.Duration'] < 5:
        risk_factor =0

    # 8.3.4: If the asset is collateralised, the risk factor is halved.
    if row['Collateralised'] == 'Yes':
        risk_factor /= 2

    # 8.3.3: SCR = Market Value × Duration × Risk Factor
    # Market value is in the 'Value' column.
    scr = row['Value'] * row['Aver.Duration'] * risk_factor
    
    return scr

# Apply the function to each row (axis=1) to calculate the SCR
df_spread_risk['SCR'] = df_spread_risk.apply(calculate_scr, axis=1)

# Display the resulting DataFrame
print("Spread Risk SCR Calculation Results:")
print(df_spread_risk.to_string())

# Calculate and display the total SCR
total_scr = df_spread_risk['SCR'].sum()
print(f"\nTotal Spread Risk SCR: £{total_scr:,.2f}")

Spread Risk SCR Calculation Results:
            Spread assets Rating  Aver.Duration Collateralised     Value        SCR
0           CCA longevity     NR           1.75             No   1440000   75600.00
1          JP Morgan Fund    AAA           1.00             No  26813000       0.00
2                 Horizon     NR           1.00             No         0       0.00
3                   Pluto     NR           1.00            Yes   1938000   29070.00
4                 Avantus     NR           1.75             No   3087000  162067.50
5                   Dayim     NR           1.75             No   5372000  282030.00
6                 Colchis     NR           1.00             No         0       0.00
7                 Hiyacar     NR           1.00             No    200000    6000.00
8                HyperJar     NR           1.00             No    972000   29160.00
9                  Ibuyer     NR           1.00             No   2599000   77970.00
10             Perceptive     NR       

In [11]:
df_spread_risk['Value'].sum()

np.int64(58143000)

#### Equity Risk
- no information in SII Assumptions Doc about how symmetric adjustment is calulated

The final capital charge for each equity is calculated using these adjusted shocks, and the results are then combined using a correlation formula to determine the overall equity risk SCR: `SCR_equity = SQRT( (SCR_type1)² + (SCR_type2)² + 2 * 0.75 * SCR_type1 * SCR_type2)`, where:
- SCR_type1: The total capital charge for all Type 1 equities.
- SCR_type2: The total capital charge for all Type 2 equities.

In [12]:
# Data from the image, using np.nan for merged/empty cells
data_equity = [
    ['Beverley Funerals', 'NR', 'Yes', 1, 4340.81, '22%', 0.000, '22.00%', 955],
    ['BVC - Fund 1', 'NR', 'No', 2, 900.18, '49%', '3.06%', '52.06%', 469],
    [np.nan, 'NR', 'No', 2, 754.22, '49%', '3.06%', '52.06%', 393],
    [np.nan, 'NR', 'No', 2, 353.89, '49%', '3.06%', '52.06%', 184],
    [np.nan, 'NR', 'No', 1, 280.92, '39%', '3.06%', '42.00%', 118],
    [np.nan, 'NR', 'No', 2, 237.24, '49%', '3.06%', '52.06%', 123],
    [np.nan, 'NR', 'No', 2, 193.10, '49%', '3.06%', '52.06%', 101],
    [np.nan, 'NR', 'No', 1, 14.13, '39%', '3.06%', '42.00%', 6],
    [np.nan, 'NR', 'No', 2, 365.26, '49%', '3.06%', '52.06%', 190],
    [np.nan, 'NR', 'No', 2, 365.26, '49%', '3.06%', '52.06%', 190],
    [np.nan, 'NR', 'No', 2, 365.26, '49%', '3.06%', '52.06%', 190],
    ['BVC - Entrepreneurs Fund', 'NR', 'No', 2, 204.72, '49%', '3.06%', '52.06%', 107],
    [np.nan, 'NR', 'No', 1, 2.06, '39%', '3.06%', '42.00%', 1],
    ['BVC - SPV 1', 'NR', 'No', 2, 625.35, '49%', '3.06%', '52.06%', 326],
    [np.nan, 'NR', 'No', 2, 631.87, '49%', '3.06%', '52.06%', 329],
    [np.nan, 'NR', 'No', 2, 84.49, '49%', '3.06%', '52.06%', 44],
    [np.nan, 'NR', 'No', 2, 1164.98, '49%', '3.06%', '52.06%', 606]
]

# Column headers from the image
columns_equity = [
    'Equity', 'Rating', 'Strategic', 'Equity Type', "Value (£'000s)",
    'Base shock (Regul.)', 'Symmetric Adj. (Dec-24)', 'Capital Charge (%)',
    "Capital Charge (£'000s)"
]

# Create the initial DataFrame
df_equity = pd.DataFrame(data_equity, columns=columns_equity)

df_equity = df_equity.ffill()

# Display the final DataFrame
print(df_equity.to_string())

                      Equity Rating Strategic  Equity Type  Value (£'000s) Base shock (Regul.) Symmetric Adj. (Dec-24) Capital Charge (%)  Capital Charge (£'000s)
0          Beverley Funerals     NR       Yes            1         4340.81                 22%                     0.0             22.00%                      955
1               BVC - Fund 1     NR        No            2          900.18                 49%                   3.06%             52.06%                      469
2               BVC - Fund 1     NR        No            2          754.22                 49%                   3.06%             52.06%                      393
3               BVC - Fund 1     NR        No            2          353.89                 49%                   3.06%             52.06%                      184
4               BVC - Fund 1     NR        No            1          280.92                 39%                   3.06%             42.00%                      118
5               BVC - 

In [13]:
df_check_equity = df_equity[['Equity', 'Rating', 'Strategic', 'Equity Type', "Value (£'000s)"]].copy()
df_check_equity['Value'] = df_check_equity["Value (£'000s)"] * 1000
df_check_equity = df_check_equity.drop(columns=["Value (£'000s)"])

In [14]:
def calculate_capital_charge(row):
    if row['Strategic'] == 'Yes':
        shock = 0.22

    else:

        if row['Equity Type'] == 1:
            shock = 0.39

        else:
            shock = 0.49

        # Symmetric adjustment - assume it is 3.06% for all
        shock += 0.0306

    capital_charge = row['Value'] * shock

    return capital_charge

# Apply the function to each row (axis=1) to calculate the SCR
df_check_equity['Capital Charge'] = df_check_equity.apply(calculate_capital_charge, axis=1)

scr_equity_1 = df_check_equity.loc[df_check_equity['Equity Type'] == 1, 'Capital Charge'].sum()
scr_equity_2 = df_check_equity.loc[df_check_equity['Equity Type'] == 2, 'Capital Charge'].sum()

equity_type_scr = np.array([scr_equity_1, scr_equity_2])
correlation_matrix_equity_type = np.array([[1,    0.75],
              [0.75, 1   ]])

variance = equity_type_scr.T @ correlation_matrix_equity_type @ equity_type_scr
scr_equity = np.sqrt(variance)

print(scr_equity)

4123866.9963296494


#### Currency Risk

- currency net assets are lower than expected (12.5M instead of 14.06M)

In [15]:
usd_bs = ['Dayim Loan (USD Purchase)', 'WSL (8VC) Shares (USD purchase)', 'CCA Longevity III DAC (USD)', 'Perceptive', 'RBS USD Account', 'JSS Forwards']
usd_list = []
for usd in usd_bs:
    usd_list.append(mi_bs.loc[mi_bs['Description']== usd, '2025-06-01 00:00:00 Actual'].item())

usd_list[-1] *= -1  # 'JSS Forwards' should be negative

currency_net_assets = sum(usd_list)

usd_forward = solvency_balance_sheet_liabilities.loc[solvency_balance_sheet_liabilities['']== 'Derivatives', 'Col N BS SII BS 2025-06-01 00:00:00'].item()

currency_risk_exposure = currency_net_assets - usd_forward

currency_scr = 0.25 * currency_risk_exposure

currency_scr_provided = scr_summary.loc[scr_summary['Item'] == 'Currency', 'Final - As at 30th June 2025'].item()
print(usd_forward)
print(currency_net_assets)
print(currency_scr, currency_scr_provided)

13895084.22
12558987.14
-334024.27 40373.94749999978


#### Concentration Risk

Use the same inputs as spread and equity risk, so inherent problems. Attempting to use figures from SII Assumptions Doc, but the Doc as some assets which were not mentioned previously appear (Secured Series Bonds A1,2,3,4,5 and AuctusLoanNotes)

All previous assets previously mentioned, which I used to try and replicate were: CCA longevity, JP Morgan Fund, Horizon, Pluto, Avantus, Dayim, Colchis, Hiyacar, HyperJar, Ibuyer, Perceptive, Wolvercote loan, Deep Discounted bonds, Beverley Funerals, BVC - Fund 1, BVC - Entrepreneurs Fund, BVC - SPV 1.

In [16]:
df_spread_risk_filtered = df_spread_risk[['Spread assets', 'Rating', 'Value']].copy()
df_check_equity_filtered = df_check_equity[['Equity', 'Rating', 'Value']].copy()

df_spread_risk_filtered.rename(columns={'Spread assets': 'Asset'}, inplace=True)
df_check_equity_filtered.rename(columns={'Equity': 'Asset'}, inplace=True)

df_assets = pd.concat([df_spread_risk_filtered, df_check_equity_filtered], ignore_index=True)

In [17]:
unrated_threshold = 0.015 * df_assets['Value'].sum()
rated_threshold = 0.3 * df_assets['Value'].sum()

def calculate_concentration(row):

    concentration_value = 0

    if row['Rating'] == 'NR':
        excess = row['Value'] - unrated_threshold

    else:
        excess = row['Value'] - rated_threshold

    if excess > 0:
        concentration_value = excess*0.73

    return concentration_value

# Apply the function to each row (axis=1) to calculate the SCR
df_assets['Concentration Value'] = df_assets.apply(calculate_concentration, axis=1)
df_assets['CV^2'] = df_assets['Concentration Value']**2
scr_concentratin = np.sqrt(df_assets['CV^2'].sum())

print(scr_concentratin)
display(df_assets)

9440386.401785271


,Asset,Rating,Value,Concentration Value,CV^2
0,CCA longevity,NR,1440000.0,295357.197,8.723587e+10
1,JP Morgan Fund,AAA,26813000.0,4456633.940,1.986159e+13
2,Horizon,NR,0.0,0.000,0.000000e+00
3,Pluto,NR,1938000.0,658897.197,4.341455e+11
4,Avantus,NR,3087000.0,1497667.197,2.243007e+12
5,Dayim,NR,5372000.0,3165717.197,1.002177e+13
6,Colchis,NR,0.0,0.000,0.000000e+00
7,Hiyacar,NR,200000.0,0.000,0.000000e+00
8,HyperJar,NR,972000.0,0.000,0.000000e+00
9,Ibuyer,NR,2599000.0,1141427.197,1.302856e+12


### Counterparty Risk

#### Counterparty type 1 (Reinsurers & Banks)

8.8.1	Type 1 counterparties are generally those with high credit quality and where the exposure is typically large but well-monitored.

-	Reinsurance recoverables: Amounts that the insurer expects to recover from its reinsurers, either due to claims already paid by the insurer or in anticipation of future claims.
-	Cash and equivalents held with banks or financial institutions.
-	Derivatives: Financial derivatives such as swaps, options, or futures where the insurer is exposed to the counterparty’s ability to honor the contract.
-	Securities lending and repurchase agreements (repos): Transactions involving temporary transfer of securities or cash.

8.8.3	Loss-given-default (LGD) for Counterparty default risk is 100% in all instances.

8.8.4	The LGD for the future derivative is `lgd_future = max(0.9 * (derivative + rm_fin) - F * collateral, 0)`, where
-	`derivative` denotes the value of the derivative
-	`rm_fin` denotes the risk mitigation effect on market risk of the derivative
-	`collateral` denotes the risk adjusted value of the collateral in relation to the derivative
-	`F` denotes a factor to take into account the economic effect of the collateral arrangement in relation to the derivative in case of a credit event related to the counterparty

8.8.5	The risk mitigation element is obtained by calculating what the SCR market risk (through currency risk) would be in the case of the forward not existing. There is no knowledge of a collateral arrangement existing for this contract.

8.8.10	The total Risk Mitigation capacity of reinsurers (RM) is calculated by reperforming the calculations for underwriting risk not factoring reinsurance recoveries, then calculate it again on a net of reinsurance basis. Risk mitigation capacity is the difference between the two underwriting charges (not factoring RI minus factoring RI).

8.8.11	Based on proportionality, the total difference between gross and net is then allocated on a pro-rata basis to the exposures by reinsurer as listed above (parag.,8.8.7) which is the simplification set out in Article 107 of the Delegated Regulations.

8.8.12	The loss-given-default (LGD) on a reinsurance arrangement: `lgd = max(0.5 * (recoverables + 0.5 * rm_re) - F * collateral, 0)`, where
- `recoverables` denotes the best estimate of amounts recoverable from the reinsurance arrangement
- `rm_re` denotes the risk mitigating effect on underwriting risk of the reinsurance arrangement
- `F` denotes a factor to take into account the economic effect of the collateral arrangement in relation to the reinsurance arrangement in case of any credit event related to the counterparty

In [18]:
counterparty_type_1_provided = scr_summary.loc[scr_summary['Item'] == 'Type 1', 'Final - As at 30th June 2025'].item()

In [34]:
# Data extracted from 8.8.7
data_reinsurers = {
    'Reinsurer': [
        'Amlin AG', 'Arch', 'Aspen', 'Axis RE', 'Everest RE', 'Lloyds', 
        'Qatar RE', 'R&V', 'SCOR', 'Tokio RE', 'Watford RE', 'Swiss RE', 
        'New RE', 'Peak RE', 'Munich Re', 'Korean Re', 'Toa Re', 'QBE', 
        'Unipol RE', 'Transatlantic RE', 'Allianz'
    ],
    'Rating': [
        'A', 'A', 'A', 'A', 'A', 'A', 'A', 'AA', 'A', 'A', 'A', 'AA', 
        'AA', 'A', 'AA', 'A', 'A', 'A', 'A', 'A', 'AA'
    ],
    'Exposure': [
        4755, 6154, 2179, 3694, 10512, 12682, 5609, 23925, 3260, 
        3659, 5592, 4161, 46440, 586, 732, 176, 146, 146, 2765, 
        18259, 13224
    ],
    'Risk Mitigating Effect': [
        1656, 2143, 759, 1287, 3662, 4418, 1954, 8334, 1136, 
        1275, 1948, 1449, 16176, 204, 255, 61, 51, 51, 963, 
        6360, 4606
    ],
    'Collateral': [
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0
    ]
}

# Create Reinsurance DataFrame
df_reinsurers = pd.DataFrame(data_reinsurers)
df_reinsurers['Exposure'] = df_reinsurers['Exposure'] * 1000
df_reinsurers['Risk Mitigating Effect'] = df_reinsurers['Risk Mitigating Effect'] * 1000

# Data extracted from 8.8.2
data_cash = {
    'Bank': [
        'RBS',
        'JSS',
        'JSS - Forward',  # This is also derivative conterparty
        'SG Hambros',
        'Barclays',
        'JP Morgan Fund - Cash',
        'HSBC'
    ],
    'Rating': [
        "A+ (S&P, Sep 2024)\nA1 (Moody's, May 2024)\nA+ (Fitch, Nov 2024)",
        "A (S&P, Jan 2024)",
        "A (S&P, Jan 2024)",
        "A (S&P, Aug 2024)\nA1 (Moody's, Jun 2024)\nA (Fitch, Oct 2024)",
        "A+ (S&P, Oct 2024)\nA1 (Moody's, March 2025)\nA+ (Fitch, July 2023)",
        "A+ (S&P, July 2024)\nAa1 (Moody's, Nov 2023)\nAA (Fitch, Dec 2024)",
        "A+ (S&P, NA)\nA1 (Moody's, NA)\nAA- (Fitch, Aug 2024)"
    ],
    'Level Selected': [
        'A (2)',
        'A (2)',
        'A (2)',
        'A (2)',
        'A (2)',
        'A (2)',
        'A (2)'
    ],
    'Exposure (£\'000s)': [
        -3212,
        6367,
        -184,
        6385,
        602,
        1115,
        126
    ],
    'Risk Mitigating Effect': [
        0, 0, 3475000, 0, 0, 0, 0
    ]
}

# Create Cash DataFrame
df_cash = pd.DataFrame(data_cash)
df_cash['Value'] = df_cash["Exposure (£'000s)"] *1000
df_cash = df_cash.drop(columns=["Exposure (£'000s)"])

In [32]:
display(df_reinsurers)
display(df_cash)

,Reinsurer,Rating,Exposure,Risk Mitigating Effect,Collateral
0,Amlin AG,A,4755000,1656000,0
1,Arch,A,6154000,2143000,0
2,Aspen,A,2179000,759000,0
3,Axis RE,A,3694000,1287000,0
4,Everest RE,A,10512000,3662000,0
5,Lloyds,A,12682000,4418000,0
6,Qatar RE,A,5609000,1954000,0
7,R&V,AA,23925000,8334000,0
8,SCOR,A,3260000,1136000,0
9,Tokio RE,A,3659000,1275000,0


,Bank,Rating,Level Selected,Risk Mitigating Effect,Value
0,RBS,"A+ (S&P, Sep 2024)\nA1 (Moody's, May 2024)\nA+...",A (2),0,-3212000
1,JSS,"A (S&P, Jan 2024)",A (2),0,6367000
2,JSS - Forward,"A (S&P, Jan 2024)",A (2),3475000,-184000
3,SG Hambros,"A (S&P, Aug 2024)\nA1 (Moody's, Jun 2024)\nA (...",A (2),0,6385000
4,Barclays,"A+ (S&P, Oct 2024)\nA1 (Moody's, March 2025)\n...",A (2),0,602000
5,JP Morgan Fund - Cash,"A+ (S&P, July 2024)\nAa1 (Moody's, Nov 2023)\n...",A (2),0,1115000
6,HSBC,"A+ (S&P, NA)\nA1 (Moody's, NA)\nAA- (Fitch, Au...",A (2),0,126000


In [35]:
F = 0

df_reinsurers['LGD'] = np.maximum(0.5 * (df_reinsurers['Exposure'] + 0.5 * df_reinsurers['Risk Mitigating Effect']) - F * df_reinsurers['Collateral'], 0)

# df_cash['LGD'] = np.maximum(0.9 * (df_cash['Value'] + df_cash['Risk Mitigating Effect']), 0)

In [39]:
df_cash_no_forward = df_cash.drop(2)
df_cash['Value'].sum()

np.int64(11199000)

#### Counterparty Type 2

"periodical aging debtors study" - source of Type 2 exposures (don't have)

8.9.1	Type 2 exposures consist of all credit exposures which are not covered in the spread risk sub-module, and which are not type 1 exposures, including the following:
-	Receivables from intermediaries
-	Policyholder debtors
-	Mortgage loans which meet the requirements in Article 191(2) to (13)
-	Deposits with ceding undertakings, where the number of single name exposures exceeds 15
-	Commitments received by an insurance or reinsurance undertaking which have been called up but are unpaid as referred to in paragraph 2(d)

8.9.2	The capital requirement for counterparty default risk on type 2 exposures shall be equal to the loss in the basic own funds that would result from an instantaneous decrease in value of type 2 exposures by the following amount: `0.9 * lgd_receivables_gre_3m + sum(0.15 * lgd)`, where:
- `lgd_receivables_gre_3m` denotes the 
- The sum is taken on all type 2 exposures other than receivables from intermediaries which have been due for more than three months.
- `lgd` denotes the loss-given-default on the type 2 exposure i

In [ ]:
counterparty_type_2_provided = scr_summary.loc[scr_summary['Item'] == 'Type 2', 'Final - As at 30th June 2025'].item()

In [ ]:
overdue_debts = 316000
standard_debts = 1840000

scr_overdue = 0.9 * overdue_debts
scr_standard = 0.15 * standard_debts

scr_type_2 = scr_overdue + scr_standard

print(scr_type_2)

560400.0


### Non-life underwriting risk

#### Non-life Premium and Reserve

8.10.1 `nlprr = 3 * theta * volume`

`volume = v_prem + v_res`

8.10.3 `v_prem = max(p_s, p_last) + fp_existing + fp_future`, where
- `p_s` has been determined based on the forecasted earned premiums in the next year per the latest business plan
- `p_last` denotes the premiums earned in the last year
- `fp_existing` denotes the premiums earned in the last year
- `fp_future` relates to premiums earning beyond 12m for BBNI policies

-	For all books, premium volumes have been selected net of reinsurance premiums in accordance with Article 116(5) of the Delegated Regulations. 
-	For books with reinsurance arrangements with counterparties in non-EEA and non-equivalent jurisdictions, reinsurance premiums can be deducted from the premium volume measures on the basis that collateral arrangements are in place in accordance with Article 213 of the Delegated Regulations.

8.10.4 `v_res = sum(res + exp)`

In [ ]:
non_life_prem_res_provided = scr_summary.loc[scr_summary['Item'] == 'Premium  & reserve risk', 'Final - As at 30th June 2025'].item()

In [ ]:
# Data extracted from 8.10.3
data_segments = {
    'HoD': [
        'Property damage (PD) & Bodily Injury (TP)',
        'Accidental damage (AD) & Windscreen damage (WS)',
        'Rescue',
        'Excess'
    ],
    'SII Segment': [
        '1 – Motor vehicle liability insurance',
        '2 – Other motor insurance',
        '8 – Assistance',
        '9 – Miscellaneous financial loss'
    ],
    'P(last, segment)': [
        8800000,
        870000,
        682000,
        79000
    ],
    'Ps': [
        10070000,
        735000,
        434000,
        72000
    ]
}

# Create Segments DataFrame
df_segments = pd.DataFrame(data_segments)

# Data extracted from 8.10.4
data_claims = {
    'HoD': [
        'Property damage (PD) & Bodily Injury (TP)',
        'Accidental damage (AD) & Windscreen damage (WS)',
        'Rescue',
        'Excess'
    ],
    'SII Segment': [
        '1 – Motor vehicle liability insurance',
        '2 – Other motor insurance',
        '8 – Assistance',
        '9 – Miscellaneous financial loss'
    ],
    'Net Discounted Claims OS & IBNR': [
        32416000,
        1131000,
        -23000,
        18000
    ],
    'Disc. RO expense provision (earned)': [
        4342000,
        138000,
        -500,
        400
    ]
}

# Create a pandas DataFrame from the dictionary
df_claims = pd.DataFrame(data_claims)

In [ ]:
fp_existing, fp_future = 0, 0  # these have no effect in the example on 8.10.3
df_segments['v_prem'] = np.maximum(df_segments['Ps'], df_segments['P(last, segment)']) + fp_existing + fp_future
df_claims['v_res'] = np.maximum(df_claims['Net Discounted Claims OS & IBNR'] + df_claims['Disc. RO expense provision (earned)'], 0)
theta = 0.0778
volume = df_claims['v_res'].sum() + df_segments['v_prem'].sum()
nlprr = 3 * theta * volume
print(nlprr)

11610809.76


#### Non-life Catastrophe

The only non-zero input is the flood SCR of £346,000 and the only applicable man-made risk is motor vehicle liability.

The £346,000 is the remaining 40% of the gross flood risk after a 60% reduction from reinsurance is applied (£866,000 * 0.40 = £346,400).

SII Assumptions Doc states the £440,000 figure is the net Solvency Capital Requirement for man-made catastrophe risk, but it does not contain the underlying data or specific calculation steps to arrive at this number.

Don't know where to source most up to date figures.

In [ ]:
non_life_catastrophe_provided = scr_summary.loc[scr_summary['Item'] == 'Cat risk', 'Final - As at 30th June 2025'].item()

In [ ]:
# Data extracted from the image
data_cat = {
    'NatCat sub-module': [
        'Windstorm',
        'Flood',
        'Earthquake',
        'Hail',
        'Subsidence'
    ],
    'SCRgross': [
        0,
        866000,
        0,
        0,
        0
    ],
    'Mitigation RI': [
        0,
        519000,
        0,
        0,
        0
    ],
    'SCRnet': [
        0,
        346000,
        0,
        0,
        0
    ]
}

# Create a pandas DataFrame
df_cat = pd.DataFrame(data_cat)

In [ ]:
df_cat['SCRnet_squared'] = df_cat['SCRnet'] ** 2
scr_nat_cat = np.sqrt(df_cat['SCRnet_squared'].sum())

scr_mm_cat = 440000  # Don't know where this comes from

scr_cat = np.sqrt(scr_nat_cat**2 + scr_mm_cat**2)
print(scr_cat)

559746.3711360709


Construct Operational

- **Inputs:**
    - **Gross Earned Premium (current year & prior year):** Sourced from `PL`.
    - **Gross Technical Provisions (non-life):** Sourced from `BS`.
    - **The BSCR:** The diversified sum of the Market, Counterparty, and Underwriting risk modules, calculated in the preceding step of the model.
    
- Calculation:
    1. **Calculate the Premium-Based Component (`Op_Prem`):** This step calculates a value based on the insurer's earned premiums and its recent growth.
        - **Formula:** `Op_Prem = 3% × GEP_CY + Max[0, 0.3% × (GEP_CY - 1.2 × GEP_PY)]`.
        - This means the model takes **3% of the gross earned premium from the last 12 months** and adds a small additional amount if premium volume has grown by more than 20% compared to the prior year, capturing the operational risk associated with rapid growth.
        
    2. **Calculate the Provisions-Based Component (`Op_Prov`):** This step calculates a value based on the size of the insurer's technical provisions.
        - **Formula:** `Op_Prov = 3% × Max(0, TP_Non-Life)`.
        - For a non-life insurer, this is **3% of the gross non-life technical provisions**.
        
    3. **Determine the Final SCR:** The model then combines these values with the BSCR.
        - **Formula:** `SCR_Operational = Min(0.3 × BSCR, Max(Op_Prem, Op_Prov))`.
        - The logic is to first take the **larger** of the premium-based and provisions-based components. This result is then compared to **30% of the BSCR**. The final Operational Risk SCR is the **smaller** of these two figures.

In [ ]:
# op_risk = op_risk_calc.loc[op_risk_calc['OP risk calc'] == 'OP risk', 'Final - As at 30th June 2025'].item()

# gep_cy = op_risk_calc.loc[op_risk_calc['OP risk calc'] == 'GEP - historic 1 to 12 months', 'Final - As at 30th June 2025'].item() # gross earned premium in the past 12 months
# gep_py = op_risk_calc.loc[op_risk_calc['OP risk calc'] == 'GEP - historic 13 - 24 months', 'Final - As at 30th June 2025'].item() # gross earned premiums in the 12 months preceding the past 12 months
# op_prem = 0.03 * gep_cy + max(0.03 * (gep_cy - 1.2 * gep_py), 0)

# tp_non_life = op_risk_calc.loc[op_risk_calc['OP risk calc'] == 'Gross TPs', 'Final - As at 30th June 2025'].item() # technical provisions recorded in the non-life underwriting module
# tp_life = 0 # technical provisions recorded in the life underwriting module
# op_prov = 0.03 * max(0, tp_non_life) + 0.045 * max(0, tp_life)

# calc_operational = min(0.3 * final_bscr, max(op_prem, op_prov))

# if math.isclose(calc_operational, op_risk):
#     print('Operational Risk Checks Out')
# else:
#     print('Operational Risk Does Not Check Out')

Source `gep_cy`, `gep_py` and `tp_non_life` myself.

In [ ]:
# mi_pl = clean_mi_pl_bs(file_path = 'documents\\Jun25 MICL MI Pack v3.1 Isi pack exc PC.xlsx', sheet_name='PL')


Gross Written Premium 1 to 12 months

In [ ]:
# gross_written_premium = mi_pl.loc[mi_pl[('', 'Description')] == 'Gross written premium', ('YTD', 'Actual')]
# gross_written_premium = gross_written_premium[:-1].item()
# change_gross_provision_unearned_premiums = mi_pl.loc[mi_pl[('', 'Description')] == 'Change in the gross provision for unearned premiums', ('YTD', 'Actual')].item()
# gross_earned_premium = gross_written_premium - change_gross_provision_unearned_premiums
# print(gross_earned_premium)

Gross Written Premium 13 to 24 months

In [ ]:
# gross_written_premium_past = mi_pl.loc[mi_pl[('', 'Description')] == 'Gross written premium', ('', 'Prior_Year')]
# gross_written_premium_past = gross_written_premium_past[:-1].item()
# change_gross_provision_unearned_premiums_past = mi_pl.loc[mi_pl[('', 'Description')] == 'Change in the gross provision for unearned premiums', ('', 'Prior_Year')].item()
# gross_earned_premium_past = gross_written_premium_past - change_gross_provision_unearned_premiums_past
# print(gross_earned_premium_past)

Gross Technical Provisions

In [ ]:
# mi_bs = clean_mi_pl_bs(file_path = 'documents\\Jun25 MICL MI Pack v3.1 Isi pack exc PC.xlsx', sheet_name='BS')

In [ ]:
# provision_for_unearned_premiums = mi_bs.loc[mi_bs[('', 'Description')] == 'Provision for unearned premiums, gross amount', ('2025-06-01 00:00:00', 'Actual')].item()
# claims_outstanding = mi_bs.loc[mi_bs[('', 'Description')] == 'Claims outstanding, gross amount', ('2025-06-01 00:00:00', 'Actual')].item()
# ibnr = mi_bs.loc[mi_bs[('', 'Description')] == 'IBNR - Gross Amount', ('2025-06-01 00:00:00', 'Actual')].item()
# gross_technical_provisions = provision_for_unearned_premiums + claims_outstanding + ibnr
# print(gross_technical_provisions)

Check to ensure calcualtion is correct

In [ ]:
def run_check(name: str, calculated_value: float, provided_value: float):
    """Compares a calculated value to a provided value and prints the result."""
    if provided_value is None:
        print(f"⚠️ {name}: Cannot check, provided value not found.")
        return

    if math.isclose(calculated_value, provided_value):
        print(f"✅ {name}: Checks out.")
    else:
        diff = abs(calculated_value - provided_value)
        # Using an f-string to format the number with commas and 2 decimal places
        print(f"❌ {name}: Does NOT check out. Difference is {diff:,.2f}.")

Function SCR Module

In [ ]:
def scr_module(correlation_matrix, risk_type, labels):

    value_column = 'Final - As at 30th June 2025'

    data = scr_summary[
        (scr_summary['SCR'] == risk_type) &
        (scr_summary['Item'].isin(labels))
    ].set_index('Item') # Set 'Item' as the index to allow ordering

    # Reindex the data to match the labels order, ensuring the vector is correct; extract the column of values and convert it to a NumPy array.
    vector = data.reindex(labels)[value_column].to_numpy()

    # vector.T @ correlation_matrix: weigh each SCR value by its correlation to all other risks & dot product with vector
    variance = vector.T @ correlation_matrix @ vector
    calc_scr = np.sqrt(variance) # bring back to original currency unit

    calc_diversification = calc_scr - sum(vector)

    validation_items = ['Total', 'Diversification']

    provided_values = scr_summary[
        (scr_summary['SCR'] == risk_type) &
        (scr_summary['Item'].isin(validation_items))
    ].set_index('Item')[value_column] # Creates a Pandas Series indexed by 'Item'

    scr_provided = provided_values.get('Total')
    diversification_scr_provided = provided_values.get('Diversification')

    print(f"--- {risk_type} Risk Validation ---")
    run_check(name=f"{risk_type} SCR",
            calculated_value=calc_scr,
            provided_value=scr_provided)

    run_check(name="Diversification Benefit",
            calculated_value=calc_diversification,
            provided_value=diversification_scr_provided)

    return calc_scr

Call SCR function

In [ ]:
A = 0.5
# Source: COMMISSION DELEGATED REGULATION (EU) 2015/35 of 10 October 2014, Article 164, Page 105 of Journal of the European Union (L 12/1)
market_scr = scr_module(
    correlation_matrix = [
    [1,    A,    A,    A,    0, 0.25],    # Interest rate
    [A,    1, 0.75, 0.75,    0, 0.25],    # Equity
    [A, 0.75,    1,  0.5,    0, 0.25],    # Property
    [A, 0.75,  0.5,    1,    0, 0.25],    # Spread
    [0,    0,    0,    0,    1,    0],     # Concentration
    [0.25, 0.25, 0.25, 0.25, 0,    1]     # Currency
], risk_type = 'Market', labels = ['Interest rate', 'Equity', 'Property', 'Spread', 'Concentration', 'Currency'])

counterparty_scr = scr_module(
    correlation_matrix = [
    [1, 0.75],  # Type 1
    [0.75, 1]  # Type 2
], risk_type = 'Counterparty', labels = ['Type 1', 'Type 2'])

nonlife_scr = scr_module(
    correlation_matrix = [
    [1.00, 0.25, 0.00],  # Premium/Reserve
    [0.25, 1.00, 0.00],  # Catastrophe
    [0.00, 0.00, 1.00]   # Lapse
], risk_type = 'Non-life', labels = ['Premium  & reserve risk', 'Cat risk', 'Lapse risk'])

--- Market Risk Validation ---
✅ Market SCR: Checks out.
✅ Diversification Benefit: Checks out.
--- Counterparty Risk Validation ---
✅ Counterparty SCR: Checks out.
✅ Diversification Benefit: Checks out.
--- Non-life Risk Validation ---
✅ Non-life SCR: Checks out.
✅ Diversification Benefit: Checks out.


BSCR

In [ ]:
labels = ['market_scr', 'counterparty_scr', 'nonlife_scr'] # set order for risks; the correlation matrix is built with this order in mind

correlation_matrix_bscr = np.array([
    [1.00, 0.25, 0.25],
    [0.25, 1.00, 0.50],
    [0.25, 0.50, 1.00]
])

scr_values = {
    'market_scr': market_scr,
    'counterparty_scr': counterparty_scr,
    'nonlife_scr': nonlife_scr
}

bscr_vector = np.array([scr_values[label] for label in labels]) # build bscr_vector in order required to match correlation matrix

# BSCR = SQRT(Σi,j Corr(i,j) * SCRi * SCRj) can be expressed in matrix notation as: BSCR = SQRT(bscr_vector^T * Correlation_Matrix * bscr_vector) where ^T denotes the transpose of the vector.
variance = bscr_vector.T @ correlation_matrix_bscr @ bscr_vector
calc_bscr = np.sqrt(variance) # brings back to original currency unit
calc_diversification = calc_bscr - sum(bscr_vector)

bscr_provided = scr_summary.loc[scr_summary['SCR'] == 'Basic SCR', 'Final - As at 30th June 2025'].item()
diversification_provided = scr_summary.loc[scr_summary['SCR'] == 'Diversification', 'Final - As at 30th June 2025'].item()

run_check(name = 'BSCR', calculated_value = calc_bscr, provided_value = bscr_provided)
run_check(name = 'BSCR Diversification', calculated_value = calc_diversification, provided_value = diversification_provided)

✅ BSCR: Checks out.
✅ BSCR Diversification: Checks out.
